In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.pyplot import *
from IPython.display import display, HTML, IFrame
%matplotlib inline
from collections import defaultdict
from IPython.display import set_matplotlib_formats
from bs4 import BeautifulSoup
set_matplotlib_formats('png','pdf')

In [2]:

brands = ['brands-data-100', 'brands-data-400', 'brands-data-it2', 'brands-data-it3']
compsci = ['compsci-data', 'compsci-data-it2', 'compsci-data-it3']
jbt = ['jbt-data', 'jbt-data-it2']
folder = compsci
MERGE_OTHER = True

In [3]:
agg_list = []
full_list = []
test_list = []
for f in folder:
    iteration = 2 if 'it3' in f else 1
    print('### {}'.format(f))
    agg =  pd.read_csv('{}/agg.csv'.format(f))
    agg['type'] = f.split('-')[0]
    agg.set_index('id')

    agg['iteration'] = iteration
    agg['source_file'] = f
    if iteration == 1:
        agg = agg.rename(columns={'label:confidence' : 'iteration_1_confidence'})
    else:
        agg = agg.rename(columns={'label:confidence' : 'iteration_2_confidence'}) 
    agg_list.append(agg)    
    
    print('Agg {}'.format(len(agg)))
   
    full = pd.read_csv('{}/full.csv'.format(f))
    full['type'] = f.split('-')[0]
    full['iteration'] = iteration
    full['source_file'] = f
    full.set_index('id')
    full_list.append( full)
    print('Full {}'.format(len(full)))
    
    test = pd.read_csv('{}/test.csv'.format(f))

    test = test[test['_hidden'] == False]
    test['type'] = f.split('-')[0]
    test.set_index('id')
    test['iteration'] = iteration
    test['source_file'] = f
    print('Test {}'.format(len(test)))
    test_list.append (test)
    
aggregated = pd.concat(agg_list)
if MERGE_OTHER:
    aggregated['label'] = aggregated.apply(lambda row: row['label'] if row['label'] != 'OTHER' else 'NONE', axis=1)
    full['label'] = full.apply(lambda row: row['label'] if row['label'] != 'OTHER' else 'NONE', axis=1)



full = pd.concat(full_list)
test_questions = pd.concat(test_list)

aggregated = aggregated[aggregated['_unit_state'] == 'finalized']
aggregated['raw_text'] = aggregated.apply(
        lambda row: BeautifulSoup(row['text_html'], "lxml").text.replace(':[OBJECT_A]', '').replace(':[OBJECT_B]', ''),
        axis=1)


def print_info(data):
    avg = pd.merge(data, full, suffixes=['_l', '_r'], on=['id'], how='left')[['id', 'text_readable_r', 'label:confidence', 'label_l', '_golden', 'label_r', '_trust','a','b']].sort_values(['label:confidence', 'label_l' ,'id'])
    renamed_avg = avg.rename(columns={'label_l' : 'assigned_by_crowdflower', 'label_r' : 'proposed_by_annotator'})
    renamed_avg
    return renamed_avg


### compsci-data
Agg 750
Full 2408
Test 53
### compsci-data-it2
Agg 1719
Full 5181
Test 56
### compsci-data-it3
Agg 2425
Full 4900
Test 58


In [10]:
a = aggregated.sort_values('_trusted_judgments',ascending=False)
print(len(a[a]))

,_golden,_last_judgment_at,_trusted_judgments,_unit_id,_unit_state,a,also_acceptable,b,expected_label,id,...,label,label_gold,label_gold_reason,marker,orig__golden,raw_text,source_file,text_html,text_readable,type
3,False,1/11/2018 13:03:12,3,1534176833,finalized,USB,NaN,Bluetooth,W,L1868520095,...,WORSE,NaN,NaN,True,NaN,"Of course you also have the option of the USB,...",compsci-data,Of course you also have the option of the <spa...,Of course you also have the option of the *USB...,compsci


In [8]:
from datetime import datetime
print(datetime.now())
print('{} {}'.format(len(full['id'].unique()),len(aggregated['id'].unique())))
dataset = pd.DataFrame(columns=['id', 'raw_text', 'better_count', 'worse_count', 'none_count', 'most_frequent_class',
                                'most_frequent_class_percentage','total_judgments', 
                                'it1_judgments', 'it2_judgments', 'it1_confidence', 'it2_confidence', 'confidence'])

for i, row in full.iterrows():
    sid = row['id']
    raw_text = row['raw_text']
    
    it1 = aggregated[(aggregated['id'] == sid) &(aggregated['iteration'] == 1)]
    it2 = aggregated[(aggregated['id'] == sid) &(aggregated['iteration'] == 2)]
    
    if len(it1)+len(it2) != 2:
        print(row['id'])
    
    if len(dataset[dataset['id'] == sid]) == 0:
        better_count = 1 if 'BETTER' in row['label'] else 0
        worse_count = 1 if 'WORSE' in row['label'] else 0
        none_count = 1 if 'NONE' in row['label'] else 0
        none_count = 1 if 'OTHER' in row['label'] else 0
        most_frequent_class = '-'
        most_frequent_class_percentage = 0
        total_judgements = 1
        it1_judgments = it1.iloc[0]['_trusted_judgments']
        it1_confidence = it1.iloc[0]['iteration_1_confidence']
        it2_judgments = it2.iloc[0]['_trusted_judgments']
        it2_confidence = it2.iloc[0]['iteration_1_confidence']

dataset

2018-03-16 19:49:15.462438
2469 2465
L1868520095


IndexError: single positional indexer is out-of-bounds

In [ ]:
from datetime import datetime
print(datetime.now())
print('{} {}'.format(len(full['id'].unique()),len(aggregated['id'].unique())))
aggregated.rename(columns={"label":"old_label"})
aggregated['better_count'] = 0
aggregated['worse_count'] = 0
aggregated['none_count'] = 0
aggregated['judgements'] = 0
aggregated['label_winner_count'] = 0
aggregated['judgements_it_1'] = 0
aggregated['judgements_it_2'] = 0
aggregated['most_frequent_class'] = ''

rnd_cnt = 0
for i, row in full.iterrows():
    try:
        r_id = row['id']

        col = row['label'].lower() + '_count'
        if col == 'other_count':
            col = 'none_count'
        
        if row['iteration'] == 1:
            aggregated.loc[aggregated['id'] == r_id, 'confidence_it_1']  aggregated['iteration_1_confidence'].value
            aggregated.loc[aggregated['id'] == r_id, 'judgements_it_1'] +=1
        else:
            #aggregated.loc[aggregated['id'] == r_id, 'confidence_it_2'] = aggregated['iteration_2_confidence'].value
            aggregated.loc[aggregated['id'] == r_id, 'judgements_it_2'] +=1
            

        aggregated.loc[aggregated['id'] == r_id, col] += 1
        aggregated.loc[aggregated['id'] == r_id, 'judgements'] += 1
        m = [
            ('WORSE', aggregated.loc[aggregated['id'] == r_id, 'worse_count'].values[0]),
            ('BETTER',  aggregated.loc[aggregated['id'] == r_id, 'better_count'].values[0]),
            ('NONE',  aggregated.loc[aggregated['id'] == r_id, 'none_count'].values[0]),
        ]
        winner = sorted(m,key=lambda k: k[1], reverse=True)
        if winner[0][1] == winner[1][1]:
            if rnd_cnt % 2 == 0:
                aggregated.loc[aggregated['id'] == r_id, 'most_frequent_class'] = winner[0][0]
                aggregated.loc[aggregated['id'] == r_id, 'most_frequent_count'] = winner[0][1]
            else:
                aggregated.loc[aggregated['id'] == r_id, 'most_frequent_class'] = winner[1][0]
                aggregated.loc[aggregated['id'] == r_id, 'most_frequent_count'] = winner[1][1]
            rnd_cnt += 1
                
        else:
            aggregated.loc[aggregated['id'] == r_id, 'most_frequent_class'] = winner[0][0]
            aggregated.loc[aggregated['id'] == r_id, 'most_frequent_count'] = winner[0][1]
    except Exception as e:
        print(r_id, e)
        
print(datetime.now())

In [ ]:
dd = aggregated.drop_duplicates(subset=['id'])


print('{} - without dups: {}'.format(len(aggregated),len(dd)))

# note: duplicates exist from the merge of the two rows (both rows still present)

dd['ratio'] =  dd['most_frequent_count']/dd['judgements']
certain = dd[dd['ratio'] > 0.5 ]
print('All: {} Certain: {}'.format(len(dd), len(dd[dd['ratio'] > 0.5 ])))
pd.set_option('display.max_colwidth', -1)

dd.sort_values('judgements')
dd['judgements'].value_counts()

### Quality

In [ ]:
conf = certain[['id','text_html', 'text_readable', 'label:confidence', 'label']]
display(conf[['label:confidence']].describe())
display(conf[['id', 'text_html']].describe())

In [ ]:
import re
from collections import defaultdict
from pprint import pprint
import operator
d = defaultdict(int)
exp = r'[\w\s+*-]+(?=:\[OBJECT_[AB]\])'
s = set()
for i, row in certain.iterrows():
    a = (sorted(re.findall(exp, row['text_html'])))
    d['{} vs. {}'.format(a[0],a[1])] += 1
print(len(d))
pprint(sorted(d.items(), key=operator.itemgetter(1)))

## Results

**Label Distribution**

In [ ]:
def make_autopct(values):
    def my_autopct(pct):
        total = sum(values)
        val = int(round(pct*total/100.0))
        return '{p:.2f}%  ({v:d})'.format(p=pct,v=val)
    return my_autopct

def show_dist(labels):

    plt.figure(figsize=plt.figaspect(1))
    plt.pie(
        labels.value_counts().values,
        labels=labels.value_counts().keys().tolist(),
        radius=2,
        autopct=make_autopct(labels.value_counts().values))
    plt.show()


**All sentences**

In [ ]:
show_dist(certain['most_frequent_class'])
show_dist(dd['most_frequent_class'])

In [ ]:
from sklearn.model_selection import train_test_split

train_i, test_i = train_test_split(dd,stratify=dd['most_frequent_class'], test_size=0.2,random_state=42)
print('{} {} {}'.format(len(train_i), len(test_i), (len(train_i)+ len(test_i))))
train_i.to_csv('data.csv')
test_i.to_csv('dev-data.csv')
print(train_i['_golden'].unique())
print(test_i['_golden'].unique())

In [ ]:
dd[['judgements']].sort_values('judgements')['judgements'].value_counts().sort_index()

In [ ]:
dd[['ratio']].sort_values('ratio')['ratio'].value_counts().sort_index()

In [ ]:
print('More than 50% {} | Less {}'.format(len(dd[dd['ratio'] >= 0.5]),len(dd[dd['ratio'] <= 0.5])))

In [ ]:
a = dd[~(dd['label'] == dd['most_frequent_class'])][['type','a','b', 'raw_text', 'better_count', 'worse_count', 'none_count', 'judgements', 'label','most_frequent_class']]
a.sort_values('judgements',ascending=False)